In [479]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
from torch.utils.data import TensorDataset, DataLoader, Subset
import random
from torchvision import models

In [480]:
data = np.load('/home/snu/Downloads/breastmnist_224.npz', allow_pickle=True)

# Combine train, val, and test sets
all_images = np.concatenate([data['train_images'], data['val_images'], data['test_images']], axis=0)
all_labels = np.concatenate([data['train_labels'], data['val_labels'], data['test_labels']], axis=0).squeeze()

In [481]:
images = all_images.astype(np.float32) / 255.0
images = np.repeat(images[:, None, :, :], 3, axis=1)  # Convert to 3 channels (N, 3, 224, 224)

# Convert to torch tensors
X = torch.tensor(images)
y = torch.tensor(all_labels).long()
print(X.shape, y.shape)

torch.Size([780, 3, 224, 224]) torch.Size([780])


In [482]:
dataset = TensorDataset(X, y)
class0_indices = [i for i in range(len(y)) if y[i] == 0]
class1_indices = [i for i in range(len(y)) if y[i] == 1]

random.seed(42)
sampled_class0 = random.sample(class0_indices, min(1000, len(class0_indices)))
sampled_class1 = random.sample(class1_indices, min(1000, len(class1_indices)))

combined_indices = sampled_class0 + sampled_class1
random.shuffle(combined_indices)

# Final subset
final_dataset = Subset(dataset, combined_indices)
final_loader = DataLoader(final_dataset, batch_size=64, shuffle=False)

In [483]:
import torch
import timm
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

vit = torch.hub.load('facebookresearch/dino:main', 'dino_vitb16')
vit.eval().to(device)

vit_feats = []
y_list = []

with torch.no_grad():
    for imgs, lbls in final_loader:
        imgs = imgs.to(device)
        feats = vit(imgs)
        vit_feats.append(feats.cpu())
        y_list.extend(lbls.cpu().tolist())

F = torch.cat(vit_feats, dim=0).numpy().astype(np.float32)
y_labels = np.array(y_list).astype(np.int64)

print("Feature shape:", F.shape)
print("Label shape:", y_labels.shape)


Using cache found in /home/snu/.cache/torch/hub/facebookresearch_dino_main


Feature shape: (780, 768)
Label shape: (780,)


In [484]:
def create_adj(F, alpha=1):
    F_norm = F / np.linalg.norm(F, axis=1, keepdims=True)
    W = np.dot(F_norm, F_norm.T)
    W = (W >= alpha).astype(np.float32)
    return W

In [485]:
def asymmetrize_random(adj_matrix, seed=None):
    """
    Randomly orient each undirected edge from a symmetric adjacency matrix.
    """
    adj = np.array(adj_matrix, dtype=np.float32)
    n = adj.shape[0]
    asym = np.zeros((n, n), dtype=np.float32)
    rng = np.random.default_rng(seed)

    for i in range(n):
        for j in range(i + 1, n):
            if adj[i, j]:
                if rng.random() < 0.5:
                    asym[i, j] = adj[i, j]
                else:
                    asym[j, i] = adj[i, j]

    return asym

In [486]:
def load_data(adj, node_feats):
    node_feats = torch.from_numpy(node_feats).float()
    edge_index = torch.from_numpy(np.array(np.nonzero(adj))).long()
    return node_feats, edge_index

In [487]:
features = F # Use ResNet embeddings

device = 'cuda' if torch.cuda.is_available() else 'cpu'

W0 = create_adj(features, alpha=0.73)
# W_asym = asymmetrize_random(W0, seed=42)
node_feats, edge_index = load_data(W0, features)
data = Data(x=node_feats, edge_index=edge_index).to(device)
A = torch.from_numpy(W0).to(device)
print(data)

Data(x=[780, 768], edge_index=[2, 266936])


In [488]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv

class GCNEncoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, device, activ=nn.ELU()):
        super(GCNEncoder, self).__init__()
        self.device = device

        self.gcn = GCNConv(input_dim, hidden_dim)
        self.bn = nn.BatchNorm1d(hidden_dim)

        self.dropout = nn.Dropout(0.3)
        self.act = activ
        self.mlp = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.gcn(x, edge_index)
        x = self.bn(x)
        x = self.act(x)
        x = self.dropout(x)

        logits = self.mlp(x)
        return logits


In [489]:
class AvgReadout(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, seq, msk=None):
        if msk is None:
            return torch.mean(seq, 0)
        else:
            msk = torch.unsqueeze(msk, -1)
            return torch.sum(seq * msk, 0) / torch.sum(msk)

In [490]:
class Discriminator(nn.Module):
    def __init__(self, n_h):
        super().__init__()
        self.f_k = nn.Bilinear(n_h, n_h, 1)
        nn.init.xavier_uniform_(self.f_k.weight.data)
        if self.f_k.bias is not None:
            self.f_k.bias.data.fill_(0.0)

    def forward(self, c, h_pl, h_mi):
        c_x = torch.unsqueeze(c, 0).expand_as(h_pl)
        sc_1 = torch.squeeze(self.f_k(h_pl, c_x), 1)
        sc_2 = torch.squeeze(self.f_k(h_mi, c_x), 1)
        logits = torch.cat((sc_1, sc_2), 0)
        return logits

In [491]:
class DGI(nn.Module):
    def __init__(self, n_in, n_h, dropout=0.25):
        super().__init__()
        self.gcn1 = GCNEncoder(n_in, n_h, device='cuda' if torch.cuda.is_available() else 'cpu', activ=nn.ELU())
        self.read = AvgReadout()
        self.sigm = nn.Sigmoid()
        self.disc = Discriminator(n_h)

    def forward(self, seq1, seq2, edge_index):
        # Create Data objects for the GCNEncoder
        data1 = Data(x=seq1, edge_index=edge_index)
        data2 = Data(x=seq2, edge_index=edge_index)

        h_1 = self.gcn1(data1)
        c = self.read(h_1)
        c = self.sigm(c)
        h_2 = self.gcn1(data2)
        logits = self.disc(c, h_1, h_2)
        return logits, h_1

In [492]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class DGI_with_classifier(DGI):
    def __init__(self, n_in, n_h, n_classes=2, cut=0, dropout=0.25):
        super().__init__(n_in, n_h, dropout=dropout)
        self.classifier = nn.Linear(n_h, n_classes)
        self.cut = cut

    def get_embeddings(self, node_feats, edge_index):
        _, embeddings = self.forward(node_feats, node_feats, edge_index)
        return embeddings

    def cut_loss(self, A, S):
        # ensure tensor
        if isinstance(A, np.ndarray):
            A = torch.from_numpy(A).float().to(S.device)

        S = F.softmax(S, dim=1)   # cluster assignment
        A_pool = torch.matmul(torch.matmul(A, S).t(), S)
        num = torch.trace(A_pool)

        D = torch.diag(torch.sum(A, dim=-1))
        D_pooled = torch.matmul(torch.matmul(D, S).t(), S)
        den = torch.trace(D_pooled)

        mincut_loss = -(num / den)

        St_S = torch.matmul(S.t(), S)
        I_S = torch.eye(S.shape[1], device=A.device)
        ortho_loss = torch.norm(St_S / torch.norm(St_S) - I_S / torch.norm(I_S))

        return mincut_loss + ortho_loss

    def modularity_loss(self, A, S):
        # ensure tensor
        if isinstance(A, np.ndarray):
            A = torch.from_numpy(A).float().to(S.device)

        C = F.softmax(S, dim=1)   # cluster assignment
        d = torch.sum(A, dim=1)
        m = torch.sum(A)

        B = A - torch.outer(d, d) / (2 * m)

        I_S = torch.eye(C.shape[1], device=A.device)
        k = torch.norm(I_S)
        n = S.shape[0]

        modularity_term = (-1 / (2 * m)) * torch.trace(torch.mm(torch.mm(C.t(), B), C))
        collapse_reg_term = (torch.sqrt(k) / n) * torch.norm(torch.sum(C, dim=0), p='fro') - 1

        return modularity_term + collapse_reg_term

    def Reg_loss(self, A, embeddings):
        # classifier output used as soft cluster assignment
        logits = self.classifier(embeddings)

        if self.cut == 1:
            return self.cut_loss(A, logits)
        else:
            return self.modularity_loss(A, logits)

In [493]:
hidden_dim = 256
cut = 0
dropout = 0.25

# make sure adjacency is a tensor on GPU
if isinstance(A, np.ndarray):
    A = torch.from_numpy(A).float().to(device)
else:
    A = A.float().to(device)

model = DGI_with_classifier(features.shape[1], hidden_dim, n_classes=2, cut=cut, dropout=dropout).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
bce_loss = nn.BCEWithLogitsLoss()

num_epochs = 2500
for epoch in range(num_epochs + 1):
    model.train()
    optimizer.zero_grad()

    perm = torch.randperm(node_feats.size(0))
    corrupt_features = node_feats[perm]

    logits, embeddings = model(node_feats.to(device), corrupt_features.to(device), edge_index.to(device))

    lbl = torch.cat([
        torch.ones(node_feats.size(0)),
        torch.zeros(node_feats.size(0))
    ]).to(device)

    dgi_loss = bce_loss(logits.squeeze(), lbl)
    reg_loss = model.Reg_loss(A, embeddings)
    loss = dgi_loss + 0.001 * reg_loss

    if epoch % 500 == 0:
        print(f"Epoch {epoch} | DGI Loss: {dgi_loss.item():.4f} | Reg Loss: {reg_loss.item():.4f} | Total: {loss.item():.4f}")

    loss.backward()
    optimizer.step()


Epoch 0 | DGI Loss: 0.7277 | Reg Loss: -0.2846 | Total: 0.7274
Epoch 500 | DGI Loss: 0.5323 | Reg Loss: -0.3100 | Total: 0.5320
Epoch 1000 | DGI Loss: 0.4259 | Reg Loss: -0.3113 | Total: 0.4256
Epoch 1500 | DGI Loss: 0.4052 | Reg Loss: -0.3109 | Total: 0.4049
Epoch 2000 | DGI Loss: 0.4008 | Reg Loss: -0.3099 | Total: 0.4005
Epoch 2500 | DGI Loss: 0.3030 | Reg Loss: -0.3084 | Total: 0.3027


In [494]:
model.eval()
with torch.no_grad():
    embeddings = model.get_embeddings(node_feats.to(device), edge_index.to(device))
    class_probabilities = F.softmax(model.classifier(embeddings), dim=1).cpu().numpy()

y_pred = np.argmax(class_probabilities, axis=1)

In [495]:
# Extract the true labels for the subset used for prediction
y_subset = y[combined_indices].cpu().numpy()

acc_score = accuracy_score(y_subset, y_pred)
acc_score_inverted = accuracy_score(y_subset, 1 - y_pred)
prec_score = precision_score(y_subset, y_pred)
rec_score = recall_score(y_subset, y_pred)
f1 = f1_score(y_subset, y_pred)
log_loss_value = log_loss(y_subset, class_probabilities)

print("Accuracy:", acc_score)
print("Accuracy (inverted):", acc_score_inverted)
print("Precision:", prec_score)
print("Recall:", rec_score)
print("F1:", f1)
print("Log Loss:", log_loss_value)

Accuracy: 0.39871794871794874
Accuracy (inverted): 0.6012820512820513
Precision: 0.6304909560723514
Recall: 0.4280701754385965
F1: 0.509926854754441
Log Loss: 1.3113071140862487


In [500]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
import torch.nn.functional as F
import torch

NUM_RUNS = 10

acc_list = []
acc_inv_list = []
prec_list = []
rec_list = []
f1_list = []
logloss_list = []

for run in range(NUM_RUNS):
    print(f"\n===== RUN {run+1}/{NUM_RUNS} =====")

    hidden_dim = 256
    cut = 0
    dropout = 0.25

    # make sure adjacency is tensor on GPU
    if isinstance(A, np.ndarray):
        A_gpu = torch.from_numpy(A).float().to(device)
    else:
        A_gpu = A.float().to(device)

    model = DGI_with_classifier(
        features.shape[1],
        hidden_dim,
        n_classes=2,
        cut=cut,
        dropout=dropout
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
    bce_loss = nn.BCEWithLogitsLoss()

    num_epochs = 2500
    for epoch in range(num_epochs + 1):
        model.train()
        optimizer.zero_grad()

        perm = torch.randperm(node_feats.size(0))
        corrupt_features = node_feats[perm]

        logits, embeddings = model(
            node_feats.to(device),
            corrupt_features.to(device),
            edge_index.to(device)
        )

        lbl = torch.cat([
            torch.ones(node_feats.size(0)),
            torch.zeros(node_feats.size(0))
        ]).to(device)

        dgi_loss = bce_loss(logits.squeeze(), lbl)
        reg_loss = model.Reg_loss(A_gpu, embeddings)
        loss = dgi_loss + 0.001 * reg_loss

        if epoch % 500 == 0:
            print(f"Epoch {epoch} | DGI Loss: {dgi_loss.item():.4f} | Reg Loss: {reg_loss.item():.4f} | Total: {loss.item():.4f}")

        loss.backward()
        optimizer.step()

    # ---------- Evaluation ----------
    model.eval()
    with torch.no_grad():
        embeddings = model.get_embeddings(node_feats.to(device), edge_index.to(device))
        class_probabilities = F.softmax(model.classifier(embeddings), dim=1).cpu().numpy()

    y_pred = np.argmax(class_probabilities, axis=1)
    y_subset = y[combined_indices].cpu().numpy()

    acc = accuracy_score(y_subset, y_pred)
    acc_inv = accuracy_score(y_subset, 1 - y_pred)
    prec = precision_score(y_subset, y_pred)
    rec = recall_score(y_subset, y_pred)
    f1 = f1_score(y_subset, y_pred)
    ll = log_loss(y_subset, class_probabilities)

    acc_list.append(acc)
    acc_inv_list.append(acc_inv)
    prec_list.append(prec)
    rec_list.append(rec)
    f1_list.append(f1)
    logloss_list.append(ll)

# -------- Print mean ± std ----------
def mean_std(arr):
    return np.mean(arr), np.std(arr)

metrics = {
    "Accuracy": acc_list,
    "Accuracy (Inverted)": acc_inv_list,
    "Precision": prec_list,
    "Recall": rec_list,
    "F1": f1_list,
    "Log Loss": logloss_list
}

print("\n===== FINAL RESULTS OVER 10 RUNS =====")
for name, values in metrics.items():
    m, s = mean_std(values)
    print(f"{name}: {m:.4f} ± {s:.4f}")



===== RUN 1/10 =====
Epoch 0 | DGI Loss: 0.7070 | Reg Loss: -0.2852 | Total: 0.7067
Epoch 500 | DGI Loss: 0.5137 | Reg Loss: -0.3077 | Total: 0.5134
Epoch 1000 | DGI Loss: 0.4295 | Reg Loss: -0.3090 | Total: 0.4292
Epoch 1500 | DGI Loss: 0.5401 | Reg Loss: -0.3099 | Total: 0.5398
Epoch 2000 | DGI Loss: 0.3254 | Reg Loss: -0.3090 | Total: 0.3251
Epoch 2500 | DGI Loss: 0.3650 | Reg Loss: -0.3082 | Total: 0.3647

===== RUN 2/10 =====
Epoch 0 | DGI Loss: 0.7048 | Reg Loss: -0.2845 | Total: 0.7045
Epoch 500 | DGI Loss: 0.5371 | Reg Loss: -0.3036 | Total: 0.5368
Epoch 1000 | DGI Loss: 0.4607 | Reg Loss: -0.3092 | Total: 0.4604
Epoch 1500 | DGI Loss: 0.4109 | Reg Loss: -0.3079 | Total: 0.4106
Epoch 2000 | DGI Loss: 0.3502 | Reg Loss: -0.3081 | Total: 0.3499
Epoch 2500 | DGI Loss: 0.4154 | Reg Loss: -0.3067 | Total: 0.4151

===== RUN 3/10 =====
Epoch 0 | DGI Loss: 0.7048 | Reg Loss: -0.2856 | Total: 0.7045
Epoch 500 | DGI Loss: 0.5256 | Reg Loss: -0.3075 | Total: 0.5253
Epoch 1000 | DGI Loss:

In [497]:
# print(y_subset)

In [498]:
# import numpy as np
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.optim import AdamW
# from torch.optim.lr_scheduler import StepLR
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss

# hidden_dim   = 256
# cut          = 0
# dropout      = 0.25
# num_runs     = 10
# num_epochs   = 8000
# lambda_list  = [0.001]
# base_seed    = 42

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# node_feats = node_feats.to(device)
# edge_index = edge_index.to(device)
# A = A.to(device)

# # Use the y_subset created earlier
# y_subset_np = y_subset.astype(int)

# N, feats_dim = node_feats.size(0), node_feats.size(1)

# all_results = []
# bce_loss = nn.BCEWithLogitsLoss()

# for lam in lambda_list:
#     print(f"\n================ LAMBDA = {lam} ================\n")

#     acc_scores, prec_scores, rec_scores, f1_scores, log_losses = [], [], [], [], []

#     for run in range(num_runs):
#         print(f"\n--- Run {run+1}/{num_runs} ---")

#         seed = base_seed + run
#         torch.manual_seed(seed)
#         np.random.seed(seed)
#         if torch.cuda.is_available():
#             torch.cuda.manual_seed_all(seed)


#         model = DGI_with_classifier(feats_dim, hidden_dim, n_classes=2, cut=cut, dropout=dropout).to(device)
#         optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0001)
#         scheduler = StepLR(optimizer, step_size=200, gamma=0.5)


#         for epoch in range(num_epochs + 1):
#             model.train()
#             optimizer.zero_grad()

#             perm = torch.randperm(N, device=device)
#             corrupt_features = node_feats[perm]

#             logits, embeddings = model(node_feats, corrupt_features, edge_index)

#             lbl = torch.cat([torch.ones(N, device=device), torch.zeros(N, device=device)])
#             dgi_loss = bce_loss(logits.squeeze(), lbl)
#             reg_loss = model.Reg_loss(A, embeddings)

#             loss = dgi_loss + lam * reg_loss

#             if epoch % 500 == 0:
#                 print(f"Epoch {epoch:4d} | DGI: {dgi_loss.item():.4f} | Reg: {reg_loss.item():.4f} | "
#                       f"λ*Reg: {(lam * reg_loss).item():.4f} | Total: {loss.item():.4f}")

#             loss.backward()
#             optimizer.step()
#             scheduler.step()

#         model.eval()
#         with torch.no_grad():
#             emb = model.get_embeddings(node_feats, edge_index)
#             logits_cls = model.classifier(emb)                   # [N, 2]
#             class_probabilities = F.softmax(logits_cls, dim=1).cpu().numpy()
#             y_pred = np.argmax(class_probabilities, axis=1)

#         acc  = accuracy_score(y_subset_np, y_pred)
#         acc_inv = accuracy_score(y_subset_np, 1 - y_pred)

#         if acc_inv > acc:
#             acc = acc_inv
#             y_pred = 1 - y_pred
#             class_probabilities = class_probabilities[:, ::-1]

#         prec = precision_score(y_subset_np, y_pred, zero_division=0)
#         rec  = recall_score(y_subset_np, y_pred, zero_division=0)
#         f1   = f1_score(y_subset_np, y_pred, zero_division=0)
#         ll   = log_loss(y_subset_np, class_probabilities)

#         print(f"Run {run+1} | Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f} | LogLoss: {ll:.4f}")

#         acc_scores.append(acc)
#         prec_scores.append(prec)
#         rec_scores.append(rec)
#         f1_scores.append(f1)
#         log_losses.append(ll)

#     lambda_results = {
#         "lambda": lam,
#         "accuracy":  (float(np.mean(acc_scores)), float(np.std(acc_scores))),
#         "precision": (float(np.mean(prec_scores)), float(np.std(prec_scores))),
#         "recall":    (float(np.mean(rec_scores)), float(np.std(rec_scores))),
#         "f1":        (float(np.mean(f1_scores)),  float(np.std(f1_scores))),
#         "log_loss":  (float(np.mean(log_losses)), float(np.std(log_losses))),
#     }
#     all_results.append(lambda_results)

#     print(f"\n--- RESULTS FOR LAMBDA = {lam} ---")
#     print(f"Accuracy : {lambda_results['accuracy'][0]:.4f} ± {lambda_results['accuracy'][1]:.4f}")
#     print(f"Precision: {lambda_results['precision'][0]:.4f} ± {lambda_results['precision'][1]:.4f}")
#     print(f"Recall   : {lambda_results['recall'][0]:.4f} ± {lambda_results['recall'][1]:.4f}")
#     print(f"F1 Score : {lambda_results['f1'][0]:.4f} ± {lambda_results['f1'][1]:.4f}")
#     print(f"Log Loss : {lambda_results['log_loss'][0]:.4f} ± {lambda_results['log_loss'][1]:.4f}")

# print("\n================ FINAL SUMMARY FOR ALL LAMBDAS ================\n")
# print(f"{'Lambda':>8} | {'Accuracy':>18} | {'Precision':>18} | {'Recall':>18} | {'F1 Score':>18} | {'Log Loss':>18}")
# print("-" * 108)
# for res in all_results:
#     print(f"{res['lambda']:>8} | "
#           f"{res['accuracy'][0]:.4f} ± {res['accuracy'][1]:.4f} | "
#           f"{res['precision'][0]:.4f} ± {res['precision'][1]:.4f} | "
#           f"{res['recall'][0]:.4f} ± {res['recall'][1]:.4f} | "
#           f"{res['f1'][0]:.4f} ± {res['f1'][1]:.4f} | "
#           f"{res['log_loss'][0]:.4f} ± {res['log_loss'][1]:.4f}")

In [499]:
# import numpy as np
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.optim import AdamW
# from torch.optim.lr_scheduler import StepLR
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss

# hidden_dim   = 256
# cut          = 0
# dropout      = 0.25
# num_runs     = 10
# num_epochs   = 8000
# lambda_list  = [0.001, 0.005, 0.009, 0.01, 0.05, 0.09, 0.1, 0.3, 0.5, 0.9, 1, 2, 5, 8]
# base_seed    = 42

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# node_feats = node_feats.to(device)
# edge_index = edge_index.to(device)
# A = A.to(device)

# # Use the y_subset created earlier
# y_subset_np = y_subset.astype(int)

# N, feats_dim = node_feats.size(0), node_feats.size(1)

# all_results = []
# bce_loss = nn.BCEWithLogitsLoss()

# for lam in lambda_list:
#     print(f"\n================ LAMBDA = {lam} ================\n")

#     acc_scores, prec_scores, rec_scores, f1_scores, log_losses = [], [], [], [], []

#     for run in range(num_runs):
#         print(f"\n--- Run {run+1}/{num_runs} ---")

#         seed = base_seed + run
#         torch.manual_seed(seed)
#         np.random.seed(seed)
#         if torch.cuda.is_available():
#             torch.cuda.manual_seed_all(seed)


#         model = DGI_with_classifier(feats_dim, hidden_dim, n_classes=2, cut=cut, dropout=dropout).to(device)
#         optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0001)
#         scheduler = StepLR(optimizer, step_size=200, gamma=0.5)


#         for epoch in range(num_epochs + 1):
#             model.train()
#             optimizer.zero_grad()

#             perm = torch.randperm(N, device=device)
#             corrupt_features = node_feats[perm]

#             logits, embeddings = model(node_feats, corrupt_features, edge_index)

#             lbl = torch.cat([torch.ones(N, device=device), torch.zeros(N, device=device)])
#             dgi_loss = bce_loss(logits.squeeze(), lbl)
#             reg_loss = model.Reg_loss(A, embeddings)

#             loss = dgi_loss + lam * reg_loss

#             if epoch % 500 == 0:
#                 print(f"Epoch {epoch:4d} | DGI: {dgi_loss.item():.4f} | Reg: {reg_loss.item():.4f} | "
#                       f"λ*Reg: {(lam * reg_loss).item():.4f} | Total: {loss.item():.4f}")

#             loss.backward()
#             optimizer.step()
#             scheduler.step()

#         model.eval()
#         with torch.no_grad():
#             emb = model.get_embeddings(node_feats, edge_index)
#             logits_cls = model.classifier(emb)                   # [N, 2]
#             class_probabilities = F.softmax(logits_cls, dim=1).cpu().numpy()
#             y_pred = np.argmax(class_probabilities, axis=1)

#         acc  = accuracy_score(y_subset_np, y_pred)
#         acc_inv = accuracy_score(y_subset_np, 1 - y_pred)

#         if acc_inv > acc:
#             acc = acc_inv
#             y_pred = 1 - y_pred
#             class_probabilities = class_probabilities[:, ::-1]

#         prec = precision_score(y_subset_np, y_pred, zero_division=0)
#         rec  = recall_score(y_subset_np, y_pred, zero_division=0)
#         f1   = f1_score(y_subset_np, y_pred, zero_division=0)
#         ll   = log_loss(y_subset_np, class_probabilities)

#         print(f"Run {run+1} | Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f} | LogLoss: {ll:.4f}")

#         acc_scores.append(acc)
#         prec_scores.append(prec)
#         rec_scores.append(rec)
#         f1_scores.append(f1)
#         log_losses.append(ll)

#     lambda_results = {
#         "lambda": lam,
#         "accuracy":  (float(np.mean(acc_scores)), float(np.std(acc_scores))),
#         "precision": (float(np.mean(prec_scores)), float(np.std(prec_scores))),
#         "recall":    (float(np.mean(rec_scores)), float(np.std(rec_scores))),
#         "f1":        (float(np.mean(f1_scores)),  float(np.std(f1_scores))),
#         "log_loss":  (float(np.mean(log_losses)), float(np.std(log_losses))),
#     }
#     all_results.append(lambda_results)

#     print(f"\n--- RESULTS FOR LAMBDA = {lam} ---")
#     print(f"Accuracy : {lambda_results['accuracy'][0]:.4f} ± {lambda_results['accuracy'][1]:.4f}")
#     print(f"Precision: {lambda_results['precision'][0]:.4f} ± {lambda_results['precision'][1]:.4f}")
#     print(f"Recall   : {lambda_results['recall'][0]:.4f} ± {lambda_results['recall'][1]:.4f}")
#     print(f"F1 Score : {lambda_results['f1'][0]:.4f} ± {lambda_results['f1'][1]:.4f}")
#     print(f"Log Loss : {lambda_results['log_loss'][0]:.4f} ± {lambda_results['log_loss'][1]:.4f}")

# print("\n================ FINAL SUMMARY FOR ALL LAMBDAS ================\n")
# print(f"{'Lambda':>8} | {'Accuracy':>18} | {'Precision':>18} | {'Recall':>18} | {'F1 Score':>18} | {'Log Loss':>18}")
# print("-" * 108)
# for res in all_results:
#     print(f"{res['lambda']:>8} | "
#           f"{res['accuracy'][0]:.4f} ± {res['accuracy'][1]:.4f} | "
#           f"{res['precision'][0]:.4f} ± {res['precision'][1]:.4f} | "
#           f"{res['recall'][0]:.4f} ± {res['recall'][1]:.4f} | "
#           f"{res['f1'][0]:.4f} ± {res['f1'][1]:.4f} | "
#           f"{res['log_loss'][0]:.4f} ± {res['log_loss'][1]:.4f}")